# 심층 신경망 훈련하기
* 심층 신경망 아래쪽으로 갈수록 그래디언트가 커지거나 줄어드는 그레디언트 소실 폭주가 있을 수 있다.
* 데이터가 충분하지 않을 수 있고 훈련이 극단적으로 느려질 수 있다.
* 수백만개의 파라미터를 가진 모델은 과적합 될 위험이 크다.
---
### 그레디언트 소실과 폭주 문제.
* 2010년 시그모이드와 가중치 초기화 방법이 문제였다는 것 발표
* 로지스틱 활성화함수는 끝에 수렴하므로 입력이 0이나 1로 수렴하면 그래디언트가 사실상 없다.
##### 글로럿과 He 초기화
* 각 층의 출력 분산이 입력분산과 같아야 각 층의 신호가 전달이 잘된다.
* 각층의 연결 가중치를 무작위로 초기화한다.
  * fan(avg)=fan(in)+fan(out) / 2
  * 평균이 0이고 분산이 1/fan(avg)인 정규분포
  * he initialization이라고도 부른다.

In [29]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

In [30]:
keras.layers.Dense(10,activation="relu",kernel_initializer="he_normal")

In [31]:
# fan(in)대신 fan(out)기반의 균등분포 he초기화를 사용하고자 하면
he_avg_init=keras.initializers.VarianceScaling(scale=2,mode='fan_avg',distribution="uniform")
keras.layers.Dense(10,activation="sigmoid",kernel_initializer=he_avg_init)

##### 수렴하지 않는 활성화 함수.
* sigmoid함수 대신 Relu함수는 특정 양숫값에 수렴하지 않는 장점이 있다.
* 뉴런이 0 이외의 값을 출력하지 않아 일부 뉴런이 죽었다는 단점이 있다.
* 이를 해결하기 위해 LeakyReLU=max(az,z)를 사용한다. az는 0 이하일때이고 a는 일반적으로 0.01을 사용한다.
* 이것이 항상 성능이 높다한다.
* 훈련중에서 a를 무작위로 선택하고 테스트 시에는 평균을 사용하는 PReLU도 성능이 꽤 높다.
* ELU도 있따 a(exp(z)-1) z<0일때.
  * 단점은 속도가 느리다!.
* 2017년 SELU활성화 함수.
  * 훈련하는 동안 각 층의 출력이 평균 0과 표준편차 1을 유지한다.
  * 깊은 네트워크에서 성능이 좋으나 조건이 있다.
    * 입력이 표준화 되어야 한다.
    * 모든 은닉층의 가중치는 르쿤 정규분포 초기화로 초기화 되어야한다. kernel_initializer="lecun_normal"
    * 일렬의 순차적 구조이어야 한다.
* 어떤 활성화 함수를 써야하는가?
  * SeLu>Elu>leakyRelu>relu>tanh>로지스틱 순으로 쓴다.
  * 자기 정규화 어려운 구조라면 Elu가 selu보다 성능이 좋고 실행속도의 측면에서 leakyRelu가 좋다.


In [32]:
(x_train_full,y_train_full),(x_test,y_test)=keras.datasets.fashion_mnist.load_data()
x_train_full=x_train_full/255
x_test=x_test/255
x_val,x_train=x_train_full[:5000],x_train_full[5000:]
y_val,y_train=y_train_full[:5000],y_train_full[5000:]

In [33]:
tf.random.set_seed(42)
np.random.seed(42)
model=keras.models.Sequential([
                               keras.layers.Flatten(input_shape=[28,28]),
                               keras.layers.Dense(300,activation="selu",kernel_initializer="lecun_normal"),# selu!
                               keras.layers.Dense(100,kernel_initializer="he_normal"),
                               keras.layers.LeakyReLU(),# 활성화함수! 리키렐루
                               keras.layers.Dense(10,activation="softmax")
])

In [34]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"]
              )

In [35]:
history=model.fit(x_train,y_train,epochs=3,batch_size=20,validation_data=(x_val,y_val))

Epoch 1/3
2750/2750 [==============================] - 7s 2ms/step - loss: 0.9179 - accuracy: 0.7103 - val_loss: 0.6519 - val_accuracy: 0.7868
Epoch 2/3
2750/2750 [==============================] - 6s 2ms/step - loss: 0.6017 - accuracy: 0.7998 - val_loss: 0.5491 - val_accuracy: 0.8202
Epoch 3/3
2750/2750 [==============================] - 6s 2ms/step - loss: 0.5326 - accuracy: 0.8201 - val_loss: 0.5142 - val_accuracy: 0.8234


##### 배치 정규화
* 그레디언트 소실과 폭주 해결을 위한 배치 정규화!
  * 각 층에서 활성화 함수를 통과하기 전이나 후에 연산을 하나 추가한다.
  * 이는 단순하게 입력을 원점에 맞추고 정규화 한 후에 각 층에서 두개의 새로운 파라미터로 결괏값의 스케일을 조정하고 이동시킨다.
  * 훈련망의 첫번째 층에 배치 정규화를 추가하면 훈련세트 표준화 할 필요가 없다.
  * 입력의 평균과 표준편차를 평가한다.
  * 성능이 아주좋다. 규제와 같은 역할을 하기에 다른 규제 기법의 필요성을 줄인다.
  * 단점으로 복잡도를 키우고 실행 시간에서 손해이다.
---
##### 케라스로 배치 정규화 구현하기.
* BatchNormalization층을 ㅜ가하면 된다.

In [36]:
model=keras.models.Sequential([
                               keras.layers.Flatten(input_shape=[28,28]),
                               keras.layers.BatchNormalization(),
                               keras.layers.Dense(300,activation="elu",kernel_initializer="he_normal"),
                               keras.layers.BatchNormalization(),
                               keras.layers.Dense(100,activation="elu",kernel_initializer="he_normal"),
                               keras.layers.BatchNormalization(),
                               keras.layers.Dense(10,activation="softmax")
])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_10 (Flatten)         (None, 784)               0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 784)               3136      
_________________________________________________________________
dense_27 (Dense)             (None, 300)               235500    
_________________________________________________________________
batch_normalization_13 (Batc (None, 300)               1200      
_________________________________________________________________
dense_28 (Dense)             (None, 100)               30100     
_________________________________________________________________
batch_normalization_14 (Batc (None, 100)               400       
_________________________________________________________________
dense_29 (Dense)             (None, 10)               

In [37]:
[(var.name,var.trainable) for var in model.layers[1].variables]# 아래 두개는 훈련되지 않는다.

[('batch_normalization_12/gamma:0', True),
 ('batch_normalization_12/beta:0', True),
 ('batch_normalization_12/moving_mean:0', False),
 ('batch_normalization_12/moving_variance:0', False)]

* 배치정규화층은 있고 없고 모두 실험해보고 어떤 것이 잘 맞는지 확인하는 것이 좋다.
* 배치 정규화 층은 입력마다 이동 파라미터를 포함하기 때문에 이전 층에서 편향을 뺄 수 있따.
* momentum 매개변수로 이동평균에 이전값 사용 가능하다.
* axis변수 중요
* 모든 층 뒤에 배치정규화 있다 가정한다. 최근 없어도 되는 가중치 초기화 기법이 나왔다.

In [38]:
model=keras.models.Sequential([
                               keras.layers.Flatten(input_shape=[28,28]),
                               keras.layers.BatchNormalization(),
                               keras.layers.Dense(300,kernel_initializer="he_normal",use_bias=False),
                               keras.layers.BatchNormalization(),
                               keras.layers.Activation("elu"),
                               keras.layers.Dense(100,kernel_initializer="he_normal",use_bias=False),
                               keras.layers.BatchNormalization(),
                               keras.layers.Activation("elu"),
                               keras.layers.Dense(10,activation="softmax")
])

In [39]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [40]:
model.fit(x_train,y_train,epochs=5,batch_size=20,validation_data=(x_val,y_val))

Epoch 1/5
2750/2750 [==============================] - 12s 4ms/step - loss: 0.5017 - accuracy: 0.8194 - val_loss: 0.3670 - val_accuracy: 0.8698
Epoch 2/5
2750/2750 [==============================] - 11s 4ms/step - loss: 0.4069 - accuracy: 0.8519 - val_loss: 0.3663 - val_accuracy: 0.8652
Epoch 3/5
2750/2750 [==============================] - 11s 4ms/step - loss: 0.3732 - accuracy: 0.8633 - val_loss: 0.3276 - val_accuracy: 0.8758
Epoch 4/5
2750/2750 [==============================] - 12s 4ms/step - loss: 0.3417 - accuracy: 0.8737 - val_loss: 0.3242 - val_accuracy: 0.8800
Epoch 5/5
2750/2750 [==============================] - 11s 4ms/step - loss: 0.3192 - accuracy: 0.8829 - val_loss: 0.3037 - val_accuracy: 0.8844


### 그레디언트 클리핑
* 그레디언트 폭주 문제 완화.
* 역전파될 때 일정 임곗값을 넘어서지 못하게 그레디언트 잘라내는 것.


In [41]:
optimizer=keras.optimizers.SGD(clipvalue=1.0)#clipnorm을 clipvalue대신 지정하면 방향을 유지한다.
model.compile(loss="mse",optimizer=optimizer)
# 그레디언트 백터의 모든 원소를 -1,1 사이로 클리핑 한다.

## 사전훈련된 층 재사용하기
* 큰 규모의 뉴럴넷을 처음부터 훈련하는 것은 좋지 않다.
* 비슷한 유형의 신경망이 있는지 찾아보고 하위층을 재사용하는 것이 좋다.
* 이를 전이학습! 이라고한다.
* ex)동물 식물 등 100개의 카테고리를 분류하는 dnn층으로 자동차 종류를 분류하기 위해서는 앞의 신경망의 일부를 재사용하는 것이 좋다.
* 상위 은닉층은 하위보다 덜 유용하다. 작업이 비슷할수록 더 많은 층을 재사용한다!. 아주 비슷하다면 출력층만 교체한다.
---
#### 케라스를 사용한 전이 학습
* A작업과 B작업이 비슷하면 A작업의 층을 기반으로 새로운 모델 만든다.

In [44]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(x_train_a, y_train_a), (x_train_b, y_train_b) = split_dataset(x_train, y_train)
(x_val_a, y_val_a), (x_val_b, y_val_b) = split_dataset(x_val, y_val)
(x_test_a, y_test_a), (x_test_b, y_test_b) = split_dataset(x_test, y_test)
x_train_b = x_train_b[:200]
y_train_b = y_train_b[:200]

In [48]:
x_train_a.shape,x_train_b.shape,y_train_a

((43986, 28, 28), (200, 28, 28), array([4, 0, 5, ..., 1, 3, 0], dtype=uint8))

In [50]:
model_a=keras.models.Sequential()
model_a.add(keras.layers.Flatten(input_shape=[28,28]))
for n_hidden in (300,100,50,50,50):
  model_a.add(keras.layers.Dense(n_hidden,activation="selu"))
model_a.add(keras.layers.Dense(8,activation="softmax"))

In [51]:
model_a.compile(loss="sparse_categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [53]:
history=model_a.fit(x_train_a,y_train_a,epochs=10,
                    batch_size=10,validation_data=(x_val_a,y_val_a))

Epoch 1/10
4399/4399 [==============================] - 13s 3ms/step - loss: 0.3699 - accuracy: 0.8706 - val_loss: 0.3853 - val_accuracy: 0.8587
Epoch 2/10
4399/4399 [==============================] - 13s 3ms/step - loss: 0.2863 - accuracy: 0.8998 - val_loss: 0.3047 - val_accuracy: 0.8994
Epoch 3/10
4399/4399 [==============================] - 13s 3ms/step - loss: 0.2546 - accuracy: 0.9100 - val_loss: 0.2399 - val_accuracy: 0.9220
Epoch 4/10
4399/4399 [==============================] - 12s 3ms/step - loss: 0.2359 - accuracy: 0.9175 - val_loss: 0.2371 - val_accuracy: 0.9205
Epoch 5/10
4399/4399 [==============================] - 13s 3ms/step - loss: 0.2219 - accuracy: 0.9226 - val_loss: 0.2633 - val_accuracy: 0.9170
Epoch 6/10
4399/4399 [==============================] - 12s 3ms/step - loss: 0.2075 - accuracy: 0.9269 - val_loss: 0.2127 - val_accuracy: 0.9320
Epoch 7/10
4399/4399 [==============================] - 12s 3ms/step - loss: 0.2016 - accuracy: 0.9290 - val_loss: 0.2302 - val_ac

In [55]:
model_a.save("my_model_a.h5")

In [62]:
model_b=keras.models.Sequential()
model_b.add(keras.layers.Flatten())
for n_hidden in (300,100,50,50,50):
  model_b.add(keras.layers.Dense(n_hidden,activation="selu"))
model_b.add(keras.layers.Dense(1,activation="sigmoid"))

In [63]:
model_b.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [64]:
model_b.fit(x_train_b,y_train_b,epochs=20,validation_data=(x_val_b,y_val_b))

Epoch 1/20
7/7 [==============================] - 1s 73ms/step - loss: 0.4304 - accuracy: 0.7800 - val_loss: 0.0669 - val_accuracy: 0.9807
Epoch 2/20
7/7 [==============================] - 0s 13ms/step - loss: 0.0212 - accuracy: 0.9950 - val_loss: 0.0981 - val_accuracy: 0.9736
Epoch 3/20
7/7 [==============================] - 0s 13ms/step - loss: 0.0089 - accuracy: 1.0000 - val_loss: 0.0976 - val_accuracy: 0.9757
Epoch 4/20
7/7 [==============================] - 0s 17ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.0829 - val_accuracy: 0.9828
Epoch 5/20
7/7 [==============================] - 0s 13ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0763 - val_accuracy: 0.9848
Epoch 6/20
7/7 [==============================] - 0s 13ms/step - loss: 6.6695e-04 - accuracy: 1.0000 - val_loss: 0.0726 - val_accuracy: 0.9848
Epoch 7/20
7/7 [==============================] - 0s 16ms/step - loss: 4.1554e-04 - accuracy: 1.0000 - val_loss: 0.0709 - val_accuracy: 0.9848
Epoch 8/20
7/7 [===

In [65]:
model_a.summary(),model_b.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_12 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_34 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_35 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_36 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_37 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_38 (Dense)             (None, 8)                

(None, None)

In [67]:
model_A=keras.models.load_model("my_model_a.h5")
model_b_on_a=keras.models.Sequential(model_A.layers[:-1])
model_b_on_a.add(keras.layers.Dense(1,activation="sigmoid"))

* model_b_on_a를 훈련할때 model_a도 영향을받는다 서로 층을 공유하기 때문에.
* 이를 원하지 않으면 model_a를 클론한다.

In [68]:
model_a_clone=keras.models.clone_model(model_A)
model_a_clone.set_weights(model_A.get_weights())

* 새로운 출력층이 랜덤하게 초기화 되어 있으므로 큰 오차를 만들것이다.
* 따라서 큰 오차 그레디언트가 재사용된 가중치를 망치지 않기 위해서 새로운 층에 적절한 가중치를 학습할 시간을 준다.
* 모든 층의 trainable 속성을 false로 지정한다.

In [69]:
for layer in model_b_on_a.layers[:-1]:
  layer.trainable=False
model_b_on_a.compile(loss="binary_crossentropy",optimizer="sgd",metrics=["accuracy"])

In [72]:
history=model_b_on_a.fit(x_train_b,y_train_b,epochs=4,validation_data=(x_val_b,y_val_b))
for layer in model_b_on_a.layers[:-1]:
  layer.trainable=True
optimizer=keras.optimizers.SGD(lr=1e-4)
model_b_on_a.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])
history=model_b_on_a.fit(x_train_b,y_train_b,epochs=16,validation_data=(x_val_b,y_val_b))

Epoch 1/4
7/7 [==============================] - 0s 20ms/step - loss: 0.1137 - accuracy: 0.9450 - val_loss: 0.1339 - val_accuracy: 0.9635
Epoch 2/4
7/7 [==============================] - 0s 12ms/step - loss: 0.0985 - accuracy: 0.9650 - val_loss: 0.1219 - val_accuracy: 0.9696
Epoch 3/4
7/7 [==============================] - 0s 11ms/step - loss: 0.0868 - accuracy: 0.9700 - val_loss: 0.1120 - val_accuracy: 0.9736
Epoch 4/4
7/7 [==============================] - 0s 15ms/step - loss: 0.0773 - accuracy: 0.9700 - val_loss: 0.1045 - val_accuracy: 0.9767


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/16
7/7 [==============================] - 1s 39ms/step - loss: 0.0720 - accuracy: 0.9750 - val_loss: 0.1042 - val_accuracy: 0.9777
Epoch 2/16
7/7 [==============================] - 0s 16ms/step - loss: 0.0716 - accuracy: 0.9750 - val_loss: 0.1039 - val_accuracy: 0.9777
Epoch 3/16
7/7 [==============================] - 0s 12ms/step - loss: 0.0712 - accuracy: 0.9800 - val_loss: 0.1036 - val_accuracy: 0.9777
Epoch 4/16
7/7 [==============================] - 0s 12ms/step - loss: 0.0709 - accuracy: 0.9800 - val_loss: 0.1034 - val_accuracy: 0.9777
Epoch 5/16
7/7 [==============================] - 0s 16ms/step - loss: 0.0706 - accuracy: 0.9800 - val_loss: 0.1031 - val_accuracy: 0.9777
Epoch 6/16
7/7 [==============================] - 0s 14ms/step - loss: 0.0702 - accuracy: 0.9800 - val_loss: 0.1029 - val_accuracy: 0.9777
Epoch 7/16
7/7 [==============================] - 0s 16ms/step - loss: 0.0699 - accuracy: 0.9800 - val_loss: 0.1026 - val_accuracy: 0.9777
Epoch 8/16
7/7 [===========

In [73]:
model_b_on_a.evaluate(x_test_b,y_test_b)

63/63 [==============================] - 0s 2ms/step - loss: 0.0734 - accuracy: 0.9865


[0.07343841344118118, 0.9865000247955322]

* 서능이 높아졌지만 전이학습은 일반적인 특성을 감지하는 경향이 있는 심층 합성곱 신경망에서 잘 작동한다.

### 비지도 사전훈련
* 레이블된 훈련이 부족할경우 비지도 사전훈련이 가능하다.
* 오토인코더나 GAN같은 훈련을 통해 하위층을 재사용하고 위에 새로운 출력층 추가한다.
* 한번에 전체 비지도 학습 모델을 훈련하고 오토 인코더나 gan을 사용.
---
### 보조 작업에서 사전 훈련
* 레이블이 부족하다면 보조작업에서 하위층을 훈련시켜 쓰는것.
* ex) 얼굴 인식의 개인별 이미지가 부족하다면 인터넷에서 무작위로 많은 인물의 이미지를 수집해서 같은 사람의 이미지인지 감지하는 첫번째 신경망을 훈련한다.
* 그 후 하위층을 재사용해 적은 양의 훈련 데이터에서 얼굴을 잘 구분하는 분류기를 훈련한다.
---
## 고속 옵티마이저
* 훈련속도를 높이는 4가지 방법. 연결 가중치에 좋은 초기화 전략 적용, 좋은 활성화 함수 사용, 배치 정규화 사용, 사전 훈련된 네트워크 재사용. 등이 있다.
* 더 빠른 최적화기도 훈련속도 높인다.
  * 모멘텀 최적화,네스테로프 가속 경사, adagrad,rmsprop,adam,nadam 등
##### 모멘텀 최적화
* 경사하강법과는 다르게 이전 그레디언트를 가속화기로 사용한다.
* 마찰값이라는 변수가 들어간다.
* 경사하강법 보다 10배 빠르게 진행된다.
* 지역 최적점을 건너뛰는데에도 도움이 된다.
---
### 네스테로프 가속 경사.
* 현제 위치가 아닌 모멘텀의 방향으로 조금 앞선 위치의 비용함수의 그래프를 계산하는 것.
* 기본 모멘텀 최적화 보다 훈련 속도가 빠르다.
  * 사용하기 위해서 SGD에 use_nesterov=True로 설정하면 된다.


In [76]:
#모멘텀
optimizer_momentum=keras.optimizers.SGD(learning_rate=0.001,momentum=0.9)
#네스테로프 가속 경사.
optimzer_nesterope=keras.optimizers.SGD(learning_rate=0.001,momentum=0.9,nesterov=True)

##### AdaGrad
* 가장 가파른 차원을 따라 그레디언트 벡터의 스케일을 감소시킨다.
* 경사가 가파른 차원에 대해 더 빠르게 감소한다.
* 적응적 학슴률이라 부르며 전역 최적점 방향으로 더 곧장 가도록 갱신한다.
* 간단한 문제에 잘 작동하지만 너무 일찍 종료되는 경우가 있기에 심층에서는 쓰지 말아야한다.
---
##### RMSProp
* 가장 최근에 반복한 그레디언트만 누적함으로 너무 빠른 수렴을 해결한다.
---
##### Adan과 NAdam
* 적응형 모멘트 추정을 의미하는 adam은 모멘텀 최적화와 RMSProp의 아이디어를 합친다.
* 하이퍼파라미터를 튜닝 할 필요가적다.
* AdaMax,Nadam등이 있다.

In [78]:
#adagrad
optimizer_adagrad=keras.optimizers.Adagrad(learning_rate=0.001)
#RMSProp
optimizer_RMSProp=keras.optimizers.RMSprop(learning_rate=0.001,rho=0.9)
#Adam
optimizer_Adam=keras.optimizers.Adam(learning_rate=0.01,beta_1=0.9,beta_2=0.999)
#Adamax
optimizer_Adamax=keras.optimizers.Adamax(learning_rate=0.001,beta_1=0.9,beta_2=0.9999)


#### 학습률 스케줄링
* 큰학습률로 시작하고 학습속도가 느려질때 학습률을 낮츠면 좋은 솔루션 발견이 쉽다.
  * 거듭제곱 기반 스케줄링-> s번 스탭을 밟을수록 학습률이 k/2,k/3방식으로 줄어든다
  * 지수 기반 스케줄링-> s스탭마다 10배씩 줌.
  * 구간별 고정 스케줄링-> 일정한 에포크동안 일정한 학습률 사용하는 방법
  * 성능 기반 스케줄링 -> 매 N스텝마다 검증오차를 측정하고 줄어들지 않으면 학습률 감소
  * 1사이클 스케줄링 -> 훈련 절반동안 학습률 선형상승하고 나머지 절반 선형 감소.마지막 몇번의 에포크는 최댓값으로 진행.

In [80]:
#거듭제곱 기반 스케줄링
optimizer_de=keras.optimizers.SGD(learning_rate=0.01,decay=1e-4)
#지수기반 스케줄링
def exponential_decay_fn(epoch):
  return 0.01*0.1**(epoch/20)
def exponential_decay(lr0,s):
  def exponential_decay_fn(epoch):
    return lr0*0.1**(epoch/20)
  return exponential_decay_fn
exponential_decay_fn=exponential_decay(lr0=0.01,s=20)

* 스케줄링 함수를 전달하여 콜백을 만들어 fit에 사용한다.
* 각 에포크 시작마다 학습률 속성을 업데이트한다.

In [83]:
lr_scheduler=keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [85]:
def piecewise_constant_fn(epoch):
  if epoch < 5:
    return 0.01
  elif epoch< 15:
    return 0.005
  else:
    return 0.001
lr_scheduler2=keras.callbacks.LearningRateScheduler(piecewise_constant_fn)
# 에포크별 학습률 지정

In [87]:
# 성능 기반 스케줄링
lr_scheduler3=keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=5)
# 일정 에포크 동안 향상되지 않을 때마다 학습률에 0.5를 곱한다.

In [89]:
s=20*len(x_train)//32 # 20번 에포크 안에 담긴 전체 스텝 수(배치크기=32)
learning_rate=keras.optimizers.schedules.ExponentialDecay(0.01,s,0.1)
optimizer=keras.optimizers.SGD(learning_rate)
# 에포크가 아닌 매 스텝마다 학습률을 업데이트 한다.

## 규제를 사용해 과대적합 피하기
* 심층신경망은 자유도가 높아 과대적합이 쉽다.
#### l1,l2규제
* 규제 사용법


In [91]:
layer=keras.layers.Dense(100,activation="elu",
                         kernel_initializer="he_normal",
                         kernel_regularizer=keras.regularizers.l1_l2(0.01))

* 코드의 가독성을 위해 함수를 감싸는 파이썬 함수

In [95]:
from functools import partial
RegularizedDense=partial(keras.layers.Dense,
                         activation="elu",
                         kernel_initializer="he_normal",
                         kernel_regularizer=keras.regularizers.l2(0.01))
model=keras.models.Sequential([
                               keras.layers.Flatten(input_shape=[28,28]),
                               keras.layers.BatchNormalization(),
                               RegularizedDense(300),
                               keras.layers.BatchNormalization(),
                               RegularizedDense(100),
                               RegularizedDense(50),
                               keras.layers.Dense(10,activation="softmax",kernel_initializer="glorot_uniform")
])
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
history=model.fit(x_train,y_train,epochs=5,batch_size=20,validation_data=(x_val,y_val))

Epoch 1/5
2750/2750 [==============================] - 14s 5ms/step - loss: 1.5846 - accuracy: 0.7743 - val_loss: 0.7001 - val_accuracy: 0.8322
Epoch 2/5
2750/2750 [==============================] - 14s 5ms/step - loss: 0.7888 - accuracy: 0.7917 - val_loss: 0.6943 - val_accuracy: 0.8214
Epoch 3/5
2750/2750 [==============================] - 13s 5ms/step - loss: 0.7253 - accuracy: 0.8060 - val_loss: 0.5965 - val_accuracy: 0.8420
Epoch 4/5
2750/2750 [==============================] - 13s 5ms/step - loss: 0.6691 - accuracy: 0.8165 - val_loss: 0.5797 - val_accuracy: 0.8478
Epoch 5/5
2750/2750 [==============================] - 13s 5ms/step - loss: 0.6338 - accuracy: 0.8230 - val_loss: 0.5450 - val_accuracy: 0.8476


##### 드롭아웃
* 아주 인기있는 규제법중 하나.
* 드롭아웃 확률 p에 따라 일정 확률이 현 스탭에서 비활성화된다.
* 보통 10~50% 사이 지정한다.
##### 알파 드롭아웃
* 출력또한 정규화에 맞춰 드롭아웃한다.
##### mc카를로 드롭아웃
* dropout을 상속해서 training=True로 설정
### 맥스-노름 규제
* 매 훈련 스텝마다 w노름을 계산해서 r보다 작도록 스케일링한다.
* r을 줄이면 규제의 양이 증가하여 과대적합을 감소시키는데 도움이 된다.

In [101]:
model=keras.models.Sequential([
                               keras.layers.Flatten(input_shape=[28,28]),
                               keras.layers.BatchNormalization(),
                               RegularizedDense(300,kernel_constraint=keras.constraints.max_norm(1.)),
                               keras.layers.BatchNormalization(),
                               keras.layers.AlphaDropout(rate=0.3),
                               RegularizedDense(100),

                               RegularizedDense(50),
                               keras.layers.Dense(10,activation="softmax",kernel_initializer="glorot_uniform")
])
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
history=model.fit(x_train,y_train,epochs=7,batch_size=20,validation_data=(x_val,y_val))

Epoch 1/7
2750/2750 [==============================] - 17s 6ms/step - loss: 1.5196 - accuracy: 0.7552 - val_loss: 0.7555 - val_accuracy: 0.8286
Epoch 2/7
2750/2750 [==============================] - 16s 6ms/step - loss: 0.8923 - accuracy: 0.7722 - val_loss: 0.8270 - val_accuracy: 0.8110
Epoch 3/7
2750/2750 [==============================] - 15s 6ms/step - loss: 0.8472 - accuracy: 0.7837 - val_loss: 0.7753 - val_accuracy: 0.8270
Epoch 4/7
2750/2750 [==============================] - 16s 6ms/step - loss: 0.8117 - accuracy: 0.7884 - val_loss: 0.6746 - val_accuracy: 0.8454
Epoch 5/7
2750/2750 [==============================] - 16s 6ms/step - loss: 0.7837 - accuracy: 0.7935 - val_loss: 0.6661 - val_accuracy: 0.8348
Epoch 6/7
2750/2750 [==============================] - 16s 6ms/step - loss: 0.7615 - accuracy: 0.7981 - val_loss: 0.8041 - val_accuracy: 0.8212
Epoch 7/7
2750/2750 [==============================] - 16s 6ms/step - loss: 0.7499 - accuracy: 0.7993 - val_loss: 0.6973 - val_accuracy: